前回の学習済みモデルから, カテゴリーに分けられた際にどの単語に注目したのかというものを見ていく. 

## 0 環境準備

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!pip install transformers fugashi ipadic
!pip install demoji
!pip install neologdn

In [4]:
!wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
!tar -zxf ldcc-20140209.tar.gz

--2024-07-05 12:53:52--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
www.rondhuit.com (www.rondhuit.com) をDNSに問いあわせています... 59.106.19.174
www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 8855190 (8.4M) [application/x-gzip]
`ldcc-20140209.tar.gz.2' に保存中

ldcc-20140209.tar.g 100%[===================>]   8.44M  2.54MB/s 時間 3.3s       

2024-07-05 12:53:56 (2.54 MB/s) - `ldcc-20140209.tar.gz.2' へ保存完了 [8855190/8855190]



## 1 データの準備(jsonファイルの作成)

In [5]:
import glob
import os
import json
import re
import demoji
import neologdn
import string

* **`glob`**: ファイルパスのパターンマッチングを行うライブラリです. 
* **`os`**: オペレーティングシステムとのインタラクションを提供するライブラリです. 
* **`json`**: JSON形式のデータの読み書きを行うライブラリです. 
* **`re`**: 正規表現処理を行うライブラリです. 
* **`demoji`**: 絵文字の処理を行うライブラリです. 
* **`neologdn`**: 現代日本語の表記ゆらぎの処理を行うライブラリです. 
* **`string`**: 文字列処理に関わる定数や関数を提供するライブラリです. 

In [6]:
path = "./text"
text_dir = os.listdir(path)
category_list = [f for f in text_dir if os.path.isdir(os.path.join(path, f))]
print(category_list)

['movie-enter', 'it-life-hack', 'kaden-channel', 'topic-news', 'livedoor-homme', 'peachy', 'sports-watch', 'dokujo-tsushin', 'smax']


In [7]:
id_category_list = []
for index, category in enumerate(category_list):
    category_dict = {"id": index, "category": category}
    id_category_list.append(category_dict)
print(id_category_list)

[{'id': 0, 'category': 'movie-enter'}, {'id': 1, 'category': 'it-life-hack'}, {'id': 2, 'category': 'kaden-channel'}, {'id': 3, 'category': 'topic-news'}, {'id': 4, 'category': 'livedoor-homme'}, {'id': 5, 'category': 'peachy'}, {'id': 6, 'category': 'sports-watch'}, {'id': 7, 'category': 'dokujo-tsushin'}, {'id': 8, 'category': 'smax'}]


In [8]:
annotations_list = []
for item in id_category_list:
    file_list = glob.glob(f'text/{item["category"]}/{item["category"]}*.txt')
    for file in file_list:
        annotation_dict = {"file_name": os.path.basename(file), "label": item["id"], "category_name": item["category"]}
        annotations_list.append(annotation_dict)
print(annotations_list[:4])

[{'file_name': 'movie-enter-6361791 3.txt', 'label': 0, 'category_name': 'movie-enter'}, {'file_name': 'movie-enter-5978741.txt', 'label': 0, 'category_name': 'movie-enter'}, {'file_name': 'movie-enter-6322901.txt', 'label': 0, 'category_name': 'movie-enter'}, {'file_name': 'movie-enter-6316535 2.txt', 'label': 0, 'category_name': 'movie-enter'}]


In [9]:
json_dict = {"category": id_category_list, "annotations": annotations_list}

In [10]:
json_save_path = "./text/dataset.json"
with open(json_save_path, mode="wt", encoding="utf-8") as f:
    json.dump(json_dict, f, indent=4)

## 2 Dataloader作成

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertJapaneseTokenizer
torch.manual_seed(6)

In [12]:
model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

In [14]:
class LivedoorDataset(Dataset):
    def __init__(self, tokenizer, text_dir=None):
        self.text_dir = text_dir
        self._load_json()
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.annotations_list)

    def __getitem__(self, idx):
        text = self._get_text(self.annotations_list[idx]["category_name"], 
                              self.annotations_list[idx]["file_name"])
        encoding = self.tokenizer(text, return_tensors="pt", max_length=512, padding="max_length", truncation=True)
        encoding = {key: torch.squeeze(value) for key, value in encoding.items()}
        encoding["labels"] = self.annotations_list[idx]["label"]

        return encoding
    
    def _load_json(self):
        with open(os.path.join(self.text_dir, 'dataset.json')) as f:
            self.text_json = json.load(f)
        self.annotations_list = self.text_json["annotations"]
        
    def _get_text(self, category_name, file_name):
        file_path = os.path.join(self.text_dir, category_name, file_name)
        lines = open(file_path).read().splitlines()
        text = '\n'.join(lines[3:]) # ファイルの4行目からを抜き出す. 
        text_preprocessed = self._text_preprocess(text)
        return text_preprocessed
        
    def _text_preprocess(self, text):
        # タブの消去
        text = text.translate(str.maketrans({'\n': '', '\t': '', '\r': '', '\u3000': ''}))

        # URLの消去
        text = re.sub(r'http?://[\w/:%#$&\?~\.=\+\-]+', '', text)
        text = re.sub(r'https?://[\w/:%#$&\?~\.=\+\-]+', '', text)

        # 絵文字の消去
        text = demoji.replace(string=text, repl='')

        # 文字の正規化
        text = neologdn.normalize(text)

        # 数字をすべて0に
        text = re.sub(r'\d+', '0', text)

        # 大文字を小文字に
        text = text.lower()

        # 【関連記事, 関連サイト, 関連リンク】以降の消去
        target_list = ['関連記事', '関連サイト', '関連リンク']
        for target in target_list:
          idx = text.find(target)
          text = text[:(idx-1)]

        return text

In [18]:
dataset = LivedoorDataset(tokenizer, "./text")
print(f"Dataset size: {len(dataset)}")
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [7000, len(dataset)-7000])
train_dataloader = DataLoader(train_dataset, batch_size=8)
val_dataloader = DataLoader(val_dataset, batch_size=8)
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Dataset size: 26933
Train dataset size: 7000
Validation dataset size: 19933


## 3 モデルのLoad

In [20]:
from transformers import BertForSequenceClassification

if torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=9).to(device)
model = torch.load("./model.pth").to(device)

mps


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


このコードは, Transformerライブラリを使用して事前学習済みのモデルをインポートし, デバイスに設定するものです. このコードは, 自然言語処理タスク, 特にBERTモデルを使用したテキスト分類タスクによく使用されます. 

**コード解説**

```python
from transformers import BertForSequenceClassification
```

この行は, Transformerライブラリから`BertForSequenceClassification`クラスをインポートします. このクラスは, BERTモデルを使用してテキスト分類タスクを実行するために使用されます. 

```python
if torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
```

この部分は, 計算に使用するデバイスを決定します. まず, `torch.backends.mps.is_available()`を使用して, AppleのMetal Performance Shaders (MPS) が利用可能かどうかを確認します. もし利用可能であれば, `device` を 'mps' に設定します. 

もしMPSが利用できない場合は, `torch.cuda.is_available()`を使用してCUDAが利用可能かどうかを確認します. CUDAが利用可能であれば, `device` を 'cuda' に設定します. 

いずれも利用できない場合は, `device` を 'cpu' に設定します. 

```python
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=9).to(device)
```

この行は, 事前学習済みのBERTモデルをインポートし, 設定します. 

- `BertForSequenceClassification.from_pretrained(model_name)`: この部分は, 指定されたモデル名 (例: "bert-base-uncased") の事前学習済みのBERTモデルをインポートします. 
- `num_labels=9`: この部分は, モデルの出力ラベルの数を9に設定します. これは, 分類タスクにおけるカテゴリの数を表します. 
- `.to(device)`: この部分は, モデルを `device` に設定します. これは, モデルの計算をCPUまたはGPUで行うことを指定します. 

```python
model = torch.load("./model.pth").to(device)
```

この行は, 保存されたモデルファイルをロードし, 設定します. 

- `torch.load("./model.pth")`: この部分は, "./model.pth" ファイルに保存されたモデルをロードします. このファイルは, 以前にトレーニングされたモデルを保存したものです. 
- `.to(device)`: この部分は, モデルを `device` に設定します. これは, モデルの計算をCPUまたはGPUで行うことを指定します. 


In [22]:
from tqdm.notebook import tqdm

model.eval()
labels_list, outputs_list = [], []

with tqdm(val_dataloader, unit="batch") as progress_bar:
  for i, batch in enumerate(progress_bar):
      batch = {key: value.to(device) for key, value in batch.items()}
      labels_list = np.concatenate([labels_list, batch["labels"].cpu().detach().numpy()])
      output = model(**batch)
      output = output.logits.argmax(axis=1).cpu().detach().numpy()
      outputs_list = np.concatenate([outputs_list, output])

accuracy = sum(outputs_list == labels_list) / len(outputs_list) * 100
print(f"accuracy: {round(accuracy, 1)}% {sum(outputs_list == labels_list)}/{len(outputs_list)}")


  0%|          | 0/2492 [00:00<?, ?batch/s]

accuracy: 94.9% 18910/19933


このコードは, バリデーションデータセットを使用してモデルの精度を評価し, 結果を出力するものです. このコードは, ニューラルネットワークモデルを評価する一般的な手法であり, 特に自然言語処理タスクにおいてよく用いられます. 

**コード解説**

```python
import numpy as np
```

この行は, NumPyライブラリをインポートします. NumPyライブラリは, 数値計算やデータ操作に役立つライブラリです. 

```python
model.eval()
```

この行は, モデルを評価モードに設定します. 評価モードでは, モデルはドロップアウトなどの学習時に行われる処理を行わず, 推論のみを行います. 

```python
labels_list, outputs_list = [], []
```

この行は, 空のリスト `labels_list` と `outputs_list` を作成します. これらのリストは, それぞれバリデーションデータセットの正解ラベルとモデルの予測ラベルを格納するために使用されます. 

```python
for i, batch in enumerate(val_dataloader):
```

この `for` ループは, バリデーションデータセットの各バッチを処理します. 

- `i`: バッチのインデックス
- `batch`: バッチデータ

```python
batch = {key: value.to(device) for key, value in batch.items()}
```

この行は, バッチ内のすべてのキーと値を `device` に転送します. これは, モデルが計算を実行するデバイス (CPU または GPU) にデータを移動することを意味します. 

```python
labels_list = np.concatenate([labels_list, batch["labels"].cpu().detach().numpy()])
```

この行は, バッチの正解ラベル (`batch["labels"]`) を `labels_list` に追加します. 

- `np.concatenate()`: NumPyライブラリの関数で, リストを連結するために使用されます. 
- `.cpu().detach().numpy()`: テンソルをCPUメモリに移動し, NumPy配列に変換します. 

```python
output = model(**batch)
```

この行は, モデルにバッチデータを入力し, モデルの出力を取得します. 

- `model(**batch)`: モデルを呼び出し, バッチデータを引数として渡します. `**` は, 辞書のキーと値をアンパックするために使用されます. 

```python
output = output.logits.argmax(axis=1).cpu().detach().numpy()
```

この行は, モデルの出力を処理し, 各サンプルに対する予測ラベルを取得します. 

- `output.logits`: モデルの出力を確率分布として表すテンソルを取得します. 
- `.argmax(axis=1)`: 各サンプルにおける確率分布の中で最も高い確率を持つインデックスを取得します (つまり, 予測ラベルを取得します). 
- `.cpu().detach().numpy()`: テンソルをCPUメモリに移動し, NumPy配列に変換します. 

```python
outputs_list = np.concatenate([outputs_list, output])
```

この行は, バッチの予測ラベル (`output`) を `outputs_list` に追加します. 

```python
accuracy = sum(outputs_list == labels_list) / len(outputs_list) * 100
```

この行は, モデルの精度を計算します. 

- `sum(outputs_list == labels_list)`: 予測ラベルと正解ラベルが一致するサンプルの数を数えます. 
- `/ len(outputs_list)`: 正解ラベルと予測ラベルのペアの総数で割ります. 
- `* 100`: 100倍して, 精度をパーセンテージで表します. 

```python
print(f"accuracy: {round(accuracy, 1)}% {sum(outputs_list == labels_list)}/{len(outputs_list)}")
```

この行は, モデルの精度をコンソールに出力します. 

- `f"accuracy: {round(accuracy, 1)}%"`: 精度を小数点第1位まで丸めてフォーマットされた文字列を出力します. 
- `{sum(outputs_list == labels_list)}/{len(outputs_list)}`: 正解ラベルと予測ラベルが一致するサンプル数とサンプルの総数をスラッシュで区切った文字列を出力します. 

## 4 Attention

In [23]:
sample_data = next(iter(val_dataloader))
sample_data = {key: value.to(device) for key, value in sample_data.items()}

labels = sample_data["labels"]
output = model(**sample_data, output_attentions=True)
pred = output[1].argmax(axis=1)
attentions = output[2]

print(f"Predicted labels: {pred.cpu().numpy()}")  # convert to numpy array for printing
print(f"True labels: {labels.cpu().numpy()}")   # convert to numpy array for printing
print(f"Attention outputs type: {type(attentions)}")
print(f"Number of attention layers: {len(attentions)}")
print(f"Last attention layer shape: {attentions[-1].shape}")

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Predicted labels: [8 8 1 5 6 3 5 2]
True labels: [8 8 1 5 6 3 5 2]
Attention outputs type: <class 'tuple'>
Number of attention layers: 12
Last attention layer shape: torch.Size([8, 12, 512, 512])


In [25]:
print(sample_data)

{'input_ids': tensor([[    2,  1751,  8810,  ..., 28550,   518,     3],
        [    2, 14486, 28548,  ...,  2161,  7686,     3],
        [    2,  4041, 28579,  ...,     0,     0,     0],
        ...,
        [    2,   518,    32,  ...,     0,     0,     0],
        [    2,  1410,   633,  ..., 15388,    16,     3],
        [    2, 25035,  3030,  ...,     0,     0,     0]], device='mps:0'), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='mps:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]], device='mps:0'), 'labels': tensor([8, 8, 1, 5, 6, 3, 5, 2], device='mps:0')}


このコードは, バリデーションデータセットからランダムに1つのサンプルデータを取得し, モデルによる予測と注意力解析を行います. このコードは, ニューラルネットワークモデルの動作を理解する上で役立ちます. 

**コード解説**

```python
sample_data = next(iter(val_dataloader))
```

この行は, バリデーションデータセットからランダムに1つのサンプルデータを取得します.  
厳密には, データを $8$ 個分とってくる.  

- `next(iter(val_dataloader))`: `val_dataloader` のイテレータから次のサンプルデータを取得します. 

```python
sample_data = {key: value.to(device) for key, value in sample_data.items()}
```

この行は, 取得したサンプルデータを `device` に転送します. これは, モデルが計算を実行するデバイス (CPU または GPU) にデータを移動することを意味します. 

```python
labels = sample_data["labels"]
```

この行は, サンプルデータの正解ラベル (`labels`) を `labels` 変数に格納します. 

```python
output = model(**sample_data, output_attentions=True)
```

この行は, モデルにサンプルデータを入力し, モデルの出力を取得します. 

- `model(**sample_data)`: モデルを呼び出し, サンプルデータを引数として渡します. `**` は, 辞書のキーと値をアンパックするために使用されます. 
- `output_attentions=True`: モデルに `output_attentions=True` オプションを渡すことで, 注意力情報も出力するように設定します. 

```python
pred = output[1].argmax(axis=1)
```

この行は, モデルの出力を処理し, 各サンプルに対する予測ラベルを取得します. 

- `output[1]`: モデルの出力の2番目の要素を取得します. これは, 通常, クラス確率分布を表します. 
- `.argmax(axis=1)`: 各サンプルにおける確率分布の中で最も高い確率を持つインデックスを取得します (つまり, 予測ラベルを取得します). 

```python
attentions = output[2]
```

この行は, モデルの出力を処理し, 注意力情報 (`attentions`) を取得します. 

- `output[2]`: モデルの出力の3番目の要素を取得します. これは, 注意力情報 (各単語に対する各ヘッドの重要度) を表します. 

```python
print(f"Predicted labels: {pred.cpu().numpy()}")
```

この行は, 予測ラベル (`pred`) をコンソールに出力します. 

- `.cpu().numpy()`: テンソルをCPUメモリに移動し, NumPy配列に変換します. 
- `f"Predicted labels: {pred.cpu().numpy()}"`: f-string を使用して, フォーマットされた文字列を出力します. 

```python
print(f"True labels: {labels.cpu().numpy()}")
```

この行は, 正解ラベル (`labels`) をコンソールに出力します. 

- `.cpu().numpy()`: テンソルをCPUメモリに移動し, NumPy配列に変換します. 
- `f"True labels: {labels.cpu().numpy()}"`: f-string を使用して, フォーマットされた文字列を出力します. 

```python
print(f"Attention outputs type: {type(attentions)}")
```

この行は, 注意力情報 (`attentions`) の型をコンソールに出力します. 

- `type(attentions)`: `attentions` の型を取得します. 
- `f"Attention outputs type: {type(attentions)}"`: f-string を使用して, フォーマットされた文字列を出力します. 

```python
print(f"Number of attention layers: {len(attentions)}")
```

この行は, 注意力情報 (`attentions`) の層数をコンソールに出力します. 

- `len(attentions)`: `attentions` のリストの長さを取得します. これは, モデルのエンコーダーの層数に相当します. 
- `f"Number of attention layers: {len(attentions)}"`: f-string を使用して, フォーマットされた文字列を出力します. 

```python
print(f"Last attention layer shape: {attentions[-1].shape}")
```

この行は, 最後の注意力情報 (`attentions[-1]`) の形状をコンソールに出力します. 

- `[8, 12, 512, 512]` は $8$ 個の記事, $12$ はヘッドの数, $512*512$ は単語のサイズ. 

**Tensor**と**Tuple**は, どちらもデータ構造を表すために用いられるプログラミング用語ですが, それぞれ異なる特徴と用途を持っています. 

### Tensor

**Tensor**は, 多次元配列を表すデータ構造です. 行列やテンソルなど, 複数の次元を持つデータを効率的に表現するために使用されます. 

**特徴:**

* 数値データだけでなく, 文字列やブール値などの非数値データも格納できます. 
* 複数の次元を持つことができ, データの形状を柔軟に定義できます. 
* 数学的な演算を効率的に実行できます. 

**主な用途:**

* 機械学習：ニューラルネットワークの入力データやモデルのパラメータなどを表すために使用されます. 
* 科学計算：物理シミュレーションや画像処理などの計算において, 多次元データを扱うために使用されます. 
* データ分析：統計分析や可視化などのデータ処理において, 多変量データを扱うために使用されます. 

**Tensorを扱うためのライブラリ:**

* NumPy：Pythonで最も一般的な数値計算ライブラリの一つであり, Tensorを扱うための基本的な機能を提供します. 
* PyTorch：機械学習に特化したPythonライブラリであり, Tensorを効率的に扱うための様々な機能を提供します. 
* TensorFlow：機械学習に特化したもう一つのPythonライブラリであり, PyTorchと同様にTensorを扱うための様々な機能を提供します. 

### Tuple

**Tuple**は, 固定長の順序付きデータ集合を表すデータ構造です. リストと似ていますが, 要素の追加や削除ができない点が異なります. 

**特徴:**

* 複数の要素を格納できますが, 要素の追加や削除はできません. 
* 要素の型は異なっていても構いません. 
* ハッシュ化可能で, キーとして使用できます. 

**主な用途:**

* 不変データの表現：変更できないデータ (設定値など) を表すために使用されます. 
* 関数の戻り値：複数の値を返す関数の戻り値として使用されます. 
* データのグループ化：関連するデータをまとめて格納するために使用されます. 

**Tupleを扱うためのライブラリ:**

* Python標準ライブラリ：Tupleを扱うための基本的な機能は, Python標準ライブラリに含まれています. 

### TensorとTupleの比較

| 項目 | Tensor | Tuple |
|---|---|---|
| データ構造 | 多次元配列 | 固定長の順序付きデータ集合 |
| 可変性 | 可変 | 不可変 |
| 要素型 | 数値データ, 文字列, ブール値など | 任意 |
| 主な用途 | 機械学習, 科学計算, データ分析 | 不変データの表現, 関数の戻り値, データのグループ化 |
| 扱うライブラリ | NumPy, PyTorch, TensorFlow | Python標準ライブラリ |

**まとめ**

* Tensorは, 多次元データを効率的に表現し, 数学的な演算を実行するために適しています. 
* Tupleは, 不変データを表現し, 複数の値を返す関数の戻り値として使用するために適しています. 


In [24]:
import textwrap

for batch_number in range(len(labels)):
  all_attens = attentions[-1][batch_number, :, 0, :].sum(axis=0)
  input_ids_index_list = all_attens.topk(20).indices
  text = tokenizer.convert_ids_to_tokens(sample_data["input_ids"][batch_number])
  for input_ids_index in input_ids_index_list:
    word = text[input_ids_index]
    text[input_ids_index] = '\033[34m' + text[input_ids_index] + '\033[0m'

  s_wrap_list = textwrap.wrap(''.join(text), 100)
  print(f"category: {category_list[labels[batch_number]]}")
  print('\n'.join(s_wrap_list), "\n")

category: smax
[CLS]n##ttドコモは0日、公式オンラインショップ「ドコモ##オン##ライン##ショップ」において端末を複数台同時購入した
場合に0台当たり最大0,0円を割り##引##く「web限定家族でキャッシュ##バック##キャン##ペーン」を0年0月0日(土)から開始することをお##知##らせしています。キャンペー
ン期間は0年0月0日(日)まで。web限定家族でキャッシュ##バック##キャン##ペーンは、現在、ドコモショップにおいて同一店舗でファミリー割引を提供している複数回線において複数台を
同時購入する際に0台当たり最大0,0円を値引く「ドコモの家族セット割」を実施していますが、これのドコモ##オン##ライン##ショップ版となります。キャンペーン内容は
、同一ファミリー割引適用内の0回線以上で対象機種に機種変更をすると、0台当たり最大0,0円が毎月の請求額からキャッシュバックされます。請求額がキャッシュバック金額に満たない場合は、差
##額分をその翌月以降に繰り##越してキャッシュバックされるということです。対象機種はドコモのスマートフォンやiモード##ケー##タイで、タブレットやデータカード、ルー##ター、フォ
トパネル以外の全機種となっています。なお、ドコモ##オン##ライン##ショップ割対象機種については、対象外となるそうです。その他の条件として、以下
のいずれかの割引サービスまたは料金プランに加入する必要があります。■f##om##aからf##om##aへの機種変の場合ファミ割m##ax0定額データスタンダード割定額データスタンダ
ード割0定額データ0k割■f##om##aからx##iへの機種変更、またはx##iからx##iへの機種変更の場合タイプx##iにねんx##iデータ##プラン##フラ##ットにねんx##iデータプランに
ねんx##iデータプラン0にねん※新規契約については、オンラインショップでお手続きと同時のファミリー割引申##込ができないため対象外となります。ご了承ください。キャンペーンを適用する
場合には、ドコモ##オン##ライン##ショップで購入時に対象条件を満たした場合に自動エントリーされるというこどで、条件さえ満た##せば自動的に適用されます。試していないので不明ですが、きちんと対象にな
る場合には購入時に

このコードは, バリデーションデータセットの各バッチについて, モデルの予測カテゴリと, その予測に最も影響を与えた単語をハイライト表示します. これは, モデルがどのように各入力テキストを理解しているのかを可視化する方法として役立ちます. 

**コード解説**

```python
import textwrap
```

この行は, `textwrap` モジュールをインポートします. このモジュールは, 長い文字列を折り返して複数行に表示するために使用されます. 

```python
for batch_number in range(len(labels)):
```

この `for` ループは, バリデーションデータセットの各バッチを処理します. 

- `batch_number`: バッチのインデックス
- `len(labels)`: バリデーションデータセットのサンプル数

```python
all_attens = attentions[-1][batch_number, :, 0, :].sum(axis=0)
```

この行は, 最後の注意力情報 (`attentions[-1]`) のバッチ `batch_number` における各単語の重要度を合計します. 

- `attentions[-1]`: 最後の注意力情報 (各単語に対する各ヘッドの重要度) を取得します. 
- `[batch_number, :, 0, :]`: バッチ `batch_number` における各単語の重要度を取得します. 
- `.sum(axis=0)`: 各単語の重要度をヘッド間で合計します. 

```python
input_ids_index_list = all_attens.topk(20).indices
```

この行は, 各単語の重要度の高い上位20個のインデックスを取得します. 

- `all_attens.topk(20)`: 各単語の重要度に基づいて上位20個の要素を取得します. 
- `.indices`: 上位20個の要素のインデックスを取得します. 

```python
text = tokenizer.convert_ids_to_tokens(sample_data["input_ids"][batch_number])
```

この行は, サンプルデータの入力テキスト (`sample_data["input_ids"][batch_number]`) をトークンに変換します. 

- `tokenizer.convert_ids_to_tokens`: トークナイザーを使用して, トークンIDをトークンに変換します. 

```python
for input_ids_index in input_ids_index_list:
    word = text[input_ids_index]
    text[input_ids_index] = '\033[34m' + text[input_ids_index] + '\033[0m'
```

この `for` ループは, 各バッチの入力テキストにおいて, 重要度の高い上位20個の単語を青色でハイライト表示します. 

- `input_ids_index`: 重要度の高い単語のインデックス
- `word`: ハイライト表示する単語
- `text[input_ids_index] = '\033[34m' + text[input_ids_index] + '\033[0m'`: ターミナルエスケープシーケンスを使用して, 単語を青色でハイライト表示します. 

```python
s_wrap_list = textwrap.wrap(''.join(text), 100)
```

この行は, ハイライト表示された入力テキストを100文字ごとに区切ってリスト (`s_wrap_list`) に格納します. 

- `textwrap.wrap(''.join(text), 100)`: 入力テキストを100文字ごとに区切ってリストに変換します. 

```python
print(f"category: {category_list[labels[batch_number]]}")
```

この行は, バッチの予測カテゴリ (`category_list[labels[batch_number]]`) をコンソールに出力します. 

```python
print('\n'.join(s_wrap_list), "\n")
```

この行は, ハイライト表示された入力テキスト (`'\n'.join(s_wrap_list)`) をコンソールに出力します. 
